In [136]:
# Check GPU
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [137]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [138]:
## Get text dataset
## Kaggle's introduction to NLP dataset
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#Unzip data
unzip_data('nlp_getting_started.zip')

--2021-11-19 19:01:58--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.147.128, 142.250.125.128, 142.250.136.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.147.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.2’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-11-19 19:01:58 (104 MB/s) - ‘nlp_getting_started.zip.2’ saved [607343/607343]



In [139]:
import pandas as pd
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [140]:
train_df.shape, test_df.shape

((7613, 5), (3263, 4))

In [141]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [142]:
# shuffle training data
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [143]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [144]:
train_df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [145]:
# number of samples
len(train_df), len(test_df)

(7613, 3263)

In [146]:
# visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f'Target: {target}', "(real diaster)" if target >0 else "(not real diaster)")
  print(f'Text:\n{text}\n')
  print('---\n')

Target: 1 (real diaster)
Text:
Top Stories - Google 4 dead dozens injured in Gaza blast near house leveled in summer warÛ_ http://t.co/P3o71DZ992

---

Target: 1 (real diaster)
Text:
Horror films are made by incredibly creative artists. We interviewed 21 of them in depth for The Anatomy of Fear. http://t.co/J6mpdsx9Lk

---

Target: 0 (not real diaster)
Text:
Kelby Tomlinson mild-mannered 2nd baseman for a great metropolitan team fights a never-ending battle for hits RBI and the #SFGiants way.

---

Target: 1 (real diaster)
Text:
@Abu_Baraa1 Suicide bomber targets Saudi mosque at least 13 dead - Suicide bomber targets Saudi mosque at least 13 dead
This is ridiculous

---

Target: 1 (real diaster)
Text:
Chances are many of us are still digging out from that catastrophe. THIS is why WoW NEEDS to evolve. THIS is why it can't be Vanilla anymore

---



In [147]:
from sklearn.model_selection import train_test_split

In [148]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size = 0.1,
                                                                            random_state = 42)

In [149]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [150]:
# first ten sentences
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [151]:
# Convert text into numbers
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [152]:
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize = "lower_and_strip_punctuation",
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens = False)

In [153]:
# Find the average number of tokens in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [154]:
# Setup text vectorization variables
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length=max_length)

In [155]:
# Fit the text vectorizer to the training set
text_vectorizer.adapt(train_sentences)

In [156]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [157]:
# random sentece from training set
random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence}\
      \n\n Vectorized version:')
text_vectorizer([random_sentence])

Original text:
 RT: fittscott: Minecraft- NIGHT LUCKY BLOCK MOD (BOB APOCALYPSE WITHER 2.0 &amp; MORE!) Mod Showcase Popularmmos: http://t.co/MuL1J9AEUx viÛ_      

 Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  96,    1, 1811,  285, 1276, 1549, 1120, 2538,  409, 2600,  693,
          35,   51, 1120, 3418]])>

In [158]:
# number of unique word in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f'Number of words in vocab: {len(words_in_vocab)}')
print(f'5 Most common words: {top_5_words}')
print(f'5 least common words: {bottom_5_words}')

Number of words in vocab: 10000
5 Most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [159]:
# Create embedding layer
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim=128,
                             input_length = max_length
                             )


In [160]:
# Get a random sentence from the taining set
random_sentence = random.choice(train_sentences)
print(f'Original text: \n {random_sentence}\
      \n \n Embedded version:')

# Embed a random sentence
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text: 
 Watch This Airport Get Swallowed Up By A Sandstorm In Under A Minute http://t.co/wD9ODwjj9L      
 
 Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-2.4543608e-02,  2.1838728e-02, -3.9946686e-02, ...,
         -2.1083839e-03,  2.2623334e-02,  4.8020016e-02],
        [ 1.5157733e-02,  1.8939003e-03, -3.1638563e-02, ...,
         -4.1377008e-02,  1.1023618e-02, -3.9445557e-02],
        [-2.0374512e-02, -3.1520054e-04,  3.1077038e-02, ...,
         -1.0053836e-02,  4.4729326e-02, -3.3665527e-02],
        ...,
        [ 1.6083155e-02, -3.6835670e-05,  3.6466327e-02, ...,
          6.4050667e-03, -1.5097428e-02, -2.6941633e-02],
        [ 8.7502375e-03, -3.3056363e-03, -4.9930133e-02, ...,
          1.2214553e-02,  5.1699392e-03,  4.2810742e-02],
        [-1.9329190e-02,  1.7828200e-02, -4.1568447e-02, ...,
         -8.8749900e-03, -4.7671236e-02, -2.1865834e-02]]], dtype=float32)>

In [161]:
# single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.02454361,  0.02183873, -0.03994669,  0.01511699,  0.02073404,
        -0.0149431 ,  0.04477241, -0.04171027,  0.0225222 ,  0.04065568,
        -0.03469146, -0.00668148,  0.04927344, -0.02046954, -0.04224679,
        -0.02872409,  0.01724337,  0.00184164,  0.01728931,  0.00215555,
         0.03866744, -0.04230641,  0.02433765,  0.00261838, -0.02894721,
         0.04972864,  0.01304034, -0.00276009,  0.02127274,  0.01270096,
         0.03504474,  0.02732643,  0.03073085, -0.03523096, -0.00759137,
         0.03865138, -0.01900011, -0.0040514 , -0.03197511,  0.01244219,
        -0.0092731 , -0.03941558, -0.03578565, -0.00704626,  0.03916876,
        -0.00120264,  0.02588135, -0.02915684, -0.01910608,  0.01993116,
         0.04915366,  0.00684275,  0.00999792,  0.02809345, -0.03915478,
         0.04165408, -0.02046197, -0.01815041, -0.03789271, -0.04874155,
        -0.01672622,  0.02469406, -0.04641199,  0.04121102,  0.04291404,
  

In [162]:
# Model 0 (Baseline model)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB()),
])

# Fit pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [163]:
# Evaluate model
baseline_score = model_0.score(val_sentences, val_labels)
print(f'Baseline model has accuracy of: {baseline_score*100:2f}%')

Baseline model has accuracy of: 79.265092%


In [164]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)

In [165]:
baseline_preds[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

In [166]:
# create an evaluation function for modelling experiments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):

  """
  Calculates model accuracy, precision, recall and f1-score for binary classification model.

  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred)*100

  # Calculate model precision, recall and f1-score using wieghted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [167]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred = baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [168]:
# Model_1: A simple dense model
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback

# Ctreate a directory to save a TensorBoard logs
SAVE_DIR = "model_logs"

In [169]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding 
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [170]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_3   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_9 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [171]:
# Compile model
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [172]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20211119-190200
Epoch 1/5
215/215 [==============================] - 4s 16ms/step - loss: 0.6114 - accuracy: 0.6960 - val_loss: 0.5357 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.4415 - accuracy: 0.8215 - val_loss: 0.4712 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3473 - accuracy: 0.8599 - val_loss: 0.4555 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2853 - accuracy: 0.8905 - val_loss: 0.4592 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2386 - accuracy: 0.9132 - val_loss: 0.4849 - val_accuracy: 0.7900


In [173]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4849 - accuracy: 0.7900


[0.48486238718032837, 0.7900262475013733]

In [174]:
model_1_pred_probs = model_1.predict(val_sentences)

In [175]:
model_1_pred_probs[0]

array([0.30038697], dtype=float32)

In [176]:
# Convert model predictions probabilities to labels
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [177]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred = model_1_preds)

model_1_results

{'accuracy': 79.00262467191601,
 'f1': 0.7861489606524887,
 'precision': 0.7980691136385264,
 'recall': 0.7900262467191601}

In [178]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [179]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [180]:
# Get the vocabulary from the text vectorization
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [181]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_3   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_9 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [201]:
# Get the weight matrix of embedding layer 
# (these are the numerical patterns between the text in the training dataset the model has learned)
embed_weights = model_1.get_layer("embedding_2").get_weights()[0]
print(embed_weights.shape) # same size as vocab size and embedding_dim (each word is a embedding_dim size vector)

(10000, 128)


In [202]:
print(embed_weights[0].shape)

(128,)


In [203]:
# Create embedding files
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [204]:
#try:
# from google.colab import files
#  files.download('vectors.tsv')
#  files.download('metadata.tsv')
#except Exception:
#  pass

In [205]:
# Create an LSTM model
from tensorflow.keras import layers

input = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
#print(x.shape)
#x = layers.LSTM(64, return_sequences=True)(x) # for stacking lSTMs
#print(x.shape)
x = layers.LSTM(64)(x)
#print(x.shape)
#x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name='model_2_LSTM')

In [206]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [207]:
# Compile the model
model_2.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

In [189]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data=(val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20211119-190245
Epoch 1/5
215/215 [==============================] - 9s 30ms/step - loss: 0.2216 - accuracy: 0.9183 - val_loss: 0.5670 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 6s 28ms/step - loss: 0.1576 - accuracy: 0.9407 - val_loss: 0.6044 - val_accuracy: 0.7913
Epoch 3/5
215/215 [==============================] - 6s 28ms/step - loss: 0.1250 - accuracy: 0.9501 - val_loss: 0.7179 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 6s 28ms/step - loss: 0.1036 - accuracy: 0.9597 - val_loss: 0.8633 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 6s 28ms/step - loss: 0.0825 - accuracy: 0.9667 - val_loss: 0.8851 - val_accuracy: 0.7730


In [190]:
# Make predictions with the LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[1.3686001e-02],
       [6.4044338e-01],
       [9.9971616e-01],
       [9.1612548e-02],
       [7.9002976e-04],
       [9.9789906e-01],
       [5.0096762e-01],
       [9.9981451e-01],
       [9.9971700e-01],
       [5.4565936e-01]], dtype=float32)

In [191]:
# Convert prediction probabilities to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [192]:
# Calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred = model_2_preds)

model_2_results

{'accuracy': 77.29658792650919,
 'f1': 0.7713337273803944,
 'precision': 0.7738445106757977,
 'recall': 0.7729658792650919}

In [193]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [194]:
## model 3: GRU (gated recurrent unit)

from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
#print(x.shape)
#x = layers.LSTM(64, return_sequences=True)(x)
#print(x.shape)
#x = layers.GRU(64)(x)
#print(x.shape)
#x = layers.Dense(64, activation='relu')(x)
#x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name='model_3_GRU')

In [195]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru_11 (GRU)                (None, 64)                37248     
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [196]:
model_3.compile(loss='binary_crossentropy',
                optimizer = 'adam',
                metrics=['accuracy'])

In [197]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20211119-190329
Epoch 1/5
215/215 [==============================] - 9s 29ms/step - loss: 0.1578 - accuracy: 0.9339 - val_loss: 0.7757 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0821 - accuracy: 0.9702 - val_loss: 0.9166 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0716 - accuracy: 0.9723 - val_loss: 0.9005 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0602 - accuracy: 0.9743 - val_loss: 0.9697 - val_accuracy: 0.7717
Epoch 5/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0548 - accuracy: 0.9774 - val_loss: 1.2396 - val_accuracy: 0.7730


In [198]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[3.5706162e-04],
       [7.6188898e-01],
       [9.9988484e-01],
       [2.9927760e-02],
       [9.9329809e-05],
       [9.9945575e-01],
       [7.3151678e-01],
       [9.9993938e-01],
       [9.9987924e-01],
       [9.7652411e-01]], dtype=float32)

In [199]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_pred_probs[:10]

array([[3.5706162e-04],
       [7.6188898e-01],
       [9.9988484e-01],
       [2.9927760e-02],
       [9.9329809e-05],
       [9.9945575e-01],
       [7.3151678e-01],
       [9.9993938e-01],
       [9.9987924e-01],
       [9.7652411e-01]], dtype=float32)

In [200]:
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)


model_3_results

{'accuracy': 77.29658792650919,
 'f1': 0.7707112186521387,
 'precision': 0.7750482481613202,
 'recall': 0.7729658792650919}

In [210]:
## Model 4: Bidirectional RNN
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
#x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name = 'model_4_bidirectional')

In [211]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_14 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [212]:
# Compile the model
model_4.compile(loss='binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

In [218]:
# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'model_4_bidirectional')])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20211119-195415
Epoch 1/5
215/215 [==============================] - 8s 38ms/step - loss: 0.0398 - accuracy: 0.9818 - val_loss: 1.6315 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 8s 37ms/step - loss: 0.0444 - accuracy: 0.9793 - val_loss: 1.3128 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 8s 37ms/step - loss: 0.0377 - accuracy: 0.9822 - val_loss: 1.4075 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 8s 38ms/step - loss: 0.0359 - accuracy: 0.9815 - val_loss: 1.8218 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 8s 37ms/step - loss: 0.0349 - accuracy: 0.9832 - val_loss: 1.3776 - val_accuracy: 0.7835


In [220]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[1.7327964e-03],
       [6.3319927e-01],
       [9.9995291e-01],
       [2.2290710e-01],
       [1.2989200e-05],
       [9.9687022e-01],
       [4.7981367e-01],
       [9.9998754e-01],
       [9.9997020e-01],
       [1.5795916e-01]], dtype=float32)

In [221]:
# Convert pred probs to pred labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 0.], dtype=float32)>

In [222]:
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred = model_4_preds)

model_4_results

{'accuracy': 78.34645669291339,
 'f1': 0.7809185675137833,
 'precision': 0.7868445599717488,
 'recall': 0.7834645669291339}

In [223]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [224]:
# Test out embedding layer, Conv1D layer and max pooling
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation='relu',
                        padding="valid")

conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPooling1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [225]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02878986, -0.02957863,  0.00413656, ..., -0.01403547,
         -0.0287559 ,  0.00395772],
        [ 0.0358575 ,  0.03283259, -0.00513042, ...,  0.0375442 ,
          0.03167171, -0.0437195 ],
        [-0.03826759,  0.01989425, -0.02592794, ..., -0.03399811,
         -0.01202602,  0.04882063],
        ...,
        [ 0.00348863, -0.01913281, -0.02576391, ...,  0.01725686,
         -0.02274721,  0.00034767],
        [ 0.00348863, -0.01913281, -0.02576391, ...,  0.01725686,
         -0.02274721,  0.00034767],
        [ 0.00348863, -0.01913281, -0.02576391, ...,  0.01725686,
         -0.02274721,  0.00034767]]], dtype=float32)>

In [226]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.02576928, 0.        , 0.        , 0.02771531, 0.        ,
         0.        , 0.        , 0.01040179, 0.03157743, 0.        ,
         0.        , 0.        , 0.02952777, 0.        , 0.0066425 ,
         0.        , 0.        , 0.03166466, 0.04841777, 0.        ,
         0.07279234, 0.03962614, 0.0184444 , 0.07497633, 0.03192649,
         0.        , 0.00766692],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.03026211, 0.04877843, 0.06522291, 0.01560479, 0.        ,
         0.03311988, 0.        , 0.        , 0.00776063, 0.03680598,
         0.        , 0.        , 0.00511147, 0.        , 0.05419542,
         0.0880457 , 0.        , 0.02854417, 0.        , 0.        ,
         0.        , 0.04090036, 0.03619181, 0.        , 0.02381439,
         0.        , 0.01303654],
        [0.        , 0.01517603, 0.        , 0.0517